In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask_ml.xgboost import XGBRegressor
from dask_ml.preprocessing import DummyEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pathlib

c:\Users\15128\anaconda3\lib\site-packages\dask_xgboost\__init__.py:7: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  warnings.warn(


In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
data = pd.read_csv('./Data/SUVData.csv')
data.drop(columns='Unnamed: 0', axis=1, inplace=True)

y = data.price
X = data.drop(['price'], axis=1)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25332 entries, 0 to 25331
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   vin                   25332 non-null  object 
 1   body_type             25332 non-null  object 
 2   city                  25332 non-null  object 
 3   city_fuel_economy     25332 non-null  bool   
 4   daysonmarket          25332 non-null  int64  
 5   dealer_zip            25332 non-null  int64  
 6   engine_cylinders      24784 non-null  object 
 7   engine_displacement   24165 non-null  float64
 8   exterior_color        25332 non-null  object 
 9   frame_damaged         25332 non-null  bool   
 10  franchise_dealer      25332 non-null  bool   
 11  franchise_make        25332 non-null  object 
 12  fuel_tank_volume      25332 non-null  object 
 13  fuel_type             25332 non-null  object 
 14  has_accidents         25332 non-null  bool   
 15  highway_fuel_econom

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [5]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

In [6]:
from sklearn.metrics import r2_score

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

my_cols = categorical_cols + numerical_cols

X_train_edit = X_train[my_cols].copy()
X_valid_edit = X_valid[my_cols].copy()

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

model = RandomForestRegressor(n_estimators=150, random_state=1)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_valid)

r2_score(y_valid, preds)

0.9186381574952138

In [8]:
import joblib 
filename = "SUV_model.pk1"
joblib.dump(pipeline, filename, compress=1)

['SUV_model.pk1']